# Prep work

In [1]:
import numpy as np
import pandas as pd
from pandas.api.types import is_numeric_dtype, is_datetime64_any_dtype
from pandas.api.types import CategoricalDtype
import scipy.fftpack
import scipy.interpolate

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
sns.set_context('notebook')
%matplotlib widget

import ipywidgets as widgets
%gui asyncio

In [2]:
df = pd.read_csv('bank-classification.csv')

# Feature engineering

It might we wise to first take a broad look at the table.

In [3]:
df.head()

,id,birth_date,job,marital,education,default,housing,loan,contact_date,contact,campaign,pdays,previous,poutcome,y
0,1,1952-03-23,housemaid,married,basic.4y,no,no,no,2008-05-12,telephone,1,999,0,nonexistent,no
1,2,1951-03-24,services,married,high.school,unknown,no,no,2008-05-26,telephone,1,999,0,nonexistent,unknown
2,3,1971-05-19,services,married,high.school,no,yes,no,2008-05-05,telephone,1,999,0,nonexistent,no
3,4,1968-01-24,admin.,married,basic.6y,no,no,no,2008-05-19,telephone,1,999,0,nonexistent,unknown
4,5,1952-05-11,services,married,high.school,no,no,yes,2008-05-19,telephone,1,999,0,nonexistent,unknown


In [4]:
df.describe(include='all', datetime_is_numeric=True)

,id,birth_date,job,marital,education,default,housing,loan,contact_date,contact,campaign,pdays,previous,poutcome,y
count,41188.00000,41188,41188,41188,41188,41188,41188,41188,41188,41188,41188.000000,41188.000000,41188.000000,41188,41188
unique,NaN,13290,12,4,8,3,3,3,552,2,NaN,NaN,NaN,3,3
top,NaN,1977-07-11,admin.,married,university.degree,no,yes,no,2008-05-21,cellular,NaN,NaN,NaN,nonexistent,unknown
freq,NaN,16,10422,24928,12168,32588,21576,33950,457,26144,NaN,NaN,NaN,35563,20389
mean,20594.50000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.567593,962.475454,0.172963,NaN,NaN
std,11890.09578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.770014,186.910907,0.494901,NaN,NaN
min,1.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,0.000000,NaN,NaN
25%,10297.75000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,999.000000,0.000000,NaN,NaN
50%,20594.50000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,999.000000,0.000000,NaN,NaN
75%,30891.25000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,999.000000,0.000000,NaN,NaN


Before we get to the more sophisticated analysis:
- `id` is not a feature, and besides *pandas* already stores it;
- we should also specify the types of features for cleaner further processing;

In [5]:
df = df.astype({
    'birth_date': 'datetime64',
    'job': 'category',
    'marital': 'category',
    'education': 'category',
    'default': 'category',
    'housing': 'category',
    'loan': 'category',
    'contact_date': 'datetime64',
    'contact': 'category',
    'campaign': 'int64',
    'pdays': 'int64',
    'previous': 'int64',
    'poutcome': 'category',
    'y': 'category'
})
df.pop('id');

## Known/Unknown
We expect that the records without `y` value were drawn uniformly from the original dataset; we will, however, need the records with known `y` value for analysis.

In [6]:
df_known = df[df['y'] != 'unknown'].copy()
yn_dt = CategoricalDtype(categories=['no', 'yes'], ordered=True)
df_known['y'] = df_known['y'].astype(yn_dt)

df_unknown = df[df['y'] == 'unknown']

## Dates
Let's start with the high-level overview.

In [7]:
date_cols = [col for col, dtype in df.dtypes.items()
                 if is_datetime64_any_dtype(dtype)]

fig, axes = plt.subplots(2, len(date_cols))

for i, col in enumerate(date_cols):
    sns.histplot(data=df_known, x=col, hue='y',
                 multiple='stack', ax=axes[0][i])
    sns.histplot(data=df_known, x=col, hue='y',
                 multiple='fill', ax=axes[1][i])
fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

So, there would, in fact, seem to be a number of correlations:
- "middle-aged" people are less likely to subscribe the term deposit;
- latter campaigns seemingly were more effective (though they also did contact less people in general).

We will add (interpolated) success rates:

In [8]:
def success_rate(col):
    samples = df[[col, 'y']]
    samples_avg = pd.crosstab(samples[col], samples['y'])
    
    avg = samples_avg['yes'] / (samples_avg['yes'] + samples_avg['no'])
    return scipy.interpolate.interp1d(samples_avg.index, avg, fill_value='extrapolate')

for col in ['contact_date', 'birth_date']:
    df['{}_D'.format(col)] = (df[col] - df[col].min()) / np.timedelta64(1, 'D')
#     df['{}_rate'.format(cof ol)] = success_rate(col)(df['{}_D'.format(col)])

We can also check whether there is any periodic pattern to the `contact_date`. Let's start by checking the success rate per day. 

In [9]:
fig, ax = plt.subplots()

days = df['contact_date_D']
full_days = np.arange(days.min(), days.max())
full_avg = success_rate('contact_date_D')(full_days)

ax.plot(full_days, full_avg)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
avg_f = scipy.fftpack.fft(full_avg)
x_f = np.linspace(0, 1, len(avg_f))
n = len(x_f)

fig, ax = plt.subplots()
ax.plot(x_f[:n//2], np.abs(avg_f[:n//2]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

So, for the most part there is no frequency (as expected), but we could add that of 1/week for good measure.

## `pdays`
If we read the description of the dataset, `pdays==999` means that there was no prior contact - let us create a new feature with that information explicitly. We shall fill the `999`s with the mean of the valid values. **Note to self: could make it a hyperparameter.**

In [12]:
df['pcontacted'] = df['pdays'] != 999
df.loc[-df['pcontacted'], 'pdays'] = df.loc[df['pcontacted'], 'pdays'].mean()

## Categorical data
Let's take a look at the categorical features.

In [1]:
cat_cols = [col for col, dtype in df.dtypes.items()
                if isinstance(dtype, CategoricalDtype)]

sel = widgets.Select(
    options=cat_cols,
    value=cat_cols[0],
    description='Columns: ',
    disabled=False
)

ok = widgets.Button(
    description='Render',
    disabled=False,
    button_style='success',
    tooltip='Render',
    icon='check'
)

display(sel, ok)

fig, (ax1, ax2) = plt.subplots(2, 1)
def render(col):
    ax1.clear()
    sns.countplot(data=df_known, x=col, hue='y',
                  ax=ax1)
    ax2.clear()
    sns.histplot(data=df_known, x=col, hue='y',
                 multiple='fill', ax=ax2)
    
    plt.show()

def on_click(_):
    ok.description = 'Rendering...'
    render(sel.value)
    ok.description = 'Render'
ok.on_click(on_click)

NameError: name 'df' is not defined

We can also take a look at the correlations between the columns:

In [15]:
F = df_known.loc[:,cat_cols].apply(lambda x: pd.factorize(x)[0])
F.corr(method='pearson')

,job,marital,education,default,housing,loan,contact,poutcome,y
job,1.000000,0.054107,0.224722,-0.110841,0.009176,-0.002210,0.074300,0.080167,0.106257
marital,0.054107,1.000000,0.039987,-0.083923,0.006507,0.000165,0.032073,0.019019,0.018560
education,0.224722,0.039987,1.000000,-0.139296,0.012397,0.006066,0.087734,0.014760,0.032035
default,-0.110841,-0.083923,-0.139296,1.000000,-0.017227,0.004623,-0.137086,-0.105978,-0.098255
housing,0.009176,0.006507,0.012397,-0.017227,1.000000,0.287551,0.056005,0.021123,0.012288
loan,-0.002210,0.000165,0.006066,0.004623,0.287551,1.000000,-0.005917,0.001633,-0.000605
contact,0.074300,0.032073,0.087734,-0.137086,0.056005,-0.005917,1.000000,0.225982,0.146894
poutcome,0.080167,0.019019,0.014760,-0.105978,0.021123,0.001633,0.225982,1.000000,0.269565
y,0.106257,0.018560,0.032035,-0.098255,0.012288,-0.000605,0.146894,0.269565,1.000000


At any rate, nothing conclusive can be gained from the analysis. I will therefore, for each categorical variable:
- add one-hot encodings thereof;
- attach a variable with success rate associated with the encoding.

In [16]:
cat_cols = [col for col, dtype in df.dtypes.items()
                if isinstance(dtype, CategoricalDtype) and col != 'y']

for col in cat_cols:
#     counts = pd.crosstab(df[col], df['y'])
#     counts_for_col = counts.loc[df[col],:]
#     rate = counts_for_col['yes'] / (counts_for_col['yes'] + counts_for_col['no'])
#     rate = pd.DataFrame(rate.values, columns=['{}__rate'.format(col)])
    
    cols = pd.get_dummies(df[col], prefix=col)    
    df = pd.concat([df, cols], axis=1)

All of this gives us following columns:

In [17]:
df.dtypes

birth_date                       datetime64[ns]
job                                    category
marital                                category
education                              category
default                                category
housing                                category
loan                                   category
contact_date                     datetime64[ns]
contact                                category
campaign                                  int64
pdays                                   float64
previous                                  int64
poutcome                               category
y                                      category
contact_date_D                          float64
birth_date_D                            float64
pcontacted                                 bool
job_admin.                                uint8
job_blue-collar                           uint8
job_entrepreneur                          uint8
job_housemaid                           

# Model

## Train/test datasets
First, we remove non-numeric values; then, we split the dataset by `y`-value.

In [18]:
def extract_num(_df):
    numeric_cols = _df.dtypes[_df.dtypes.apply(is_numeric_dtype)]
    return _df[numeric_cols.index].astype('float64')

features = df.copy()
labels = features.pop('y')

train_Ix = (labels != 'unknown')

train_features = features[train_Ix]
train_features = extract_num(train_features)

train_labels = labels[train_Ix]
train_labels = pd.DataFrame(train_labels.astype(yn_dt).cat.codes,
                            columns=['y'])

test_Ix = (labels == 'unknown')
test_features = features[test_Ix]
test_features = extract_num(test_features)

Let us also define some utility functions for further splitting.

In [51]:
from sklearn.model_selection import train_test_split

def _choose_Ix(train, n):
    train_n = int(np.ceil(train*n))
    train_Ix = np.random.choice(n, train_n, replace=False)
    test_Ix = np.setdiff1d(np.arange(n), train_Ix)
    return train_Ix, test_Ix

def split(train, X=train_features, y=train_labels):
    train_Ix, test_Ix = _choose_Ix(train, X.shape[0])
    np.random.shuffle(train_Ix)
    np.random.shuffle(test_Ix)    
    
    return X.iloc[train_Ix,:], y.iloc[train_Ix,:],\
           X.iloc[test_Ix,:], y.iloc[test_Ix,:]

def choose(frac, X=train_features, y=train_labels):
    train_Ix, _ = _choose_Ix(frac, X.shape[0])
    np.random.shuffle(train_Ix)
    return X.iloc[train_Ix,:], y.iloc[train_Ix,:]    

## Model № 1 (Sklearn)

In [20]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, RandomizedSearchCV,\
    GridSearchCV
import tempfile

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import SGDClassifier, RidgeClassifierCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import\
    AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier,\
    StackingClassifier, HistGradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import SelectKBest, chi2, SelectFromModel

from xgboost import XGBClassifier

We will essentially stack a bunch of various (sometimes boosting) models; we will also attach hyperparameters for use in the search.

In [21]:
base = {
#     'svc': {
#         'clf': SVC(kernel='rbf', C=1),
#         'grid': {
#             'kernel': ['rbf', 'poly', 'sigmoid', 'linear'],
#             'C': [0.1, 1, 10, 40, 100],
#             'gamma': ['auto', 1, 0.1, 0.01]
#         }
#     },
    'rf': {
        'clf': RandomForestClassifier(),
        'grid': {
            'max_features': ['auto', 'sqrt', 'log2'],
            'n_estimators': [10, 100, 250, 1000],
        }
    },
    'et': {
        'clf': ExtraTreesClassifier(),
        'grid': {
            'n_estimators': [*range(50, 250+1, 50), 1000],
            'max_features': ['auto', 'sqrt', 'log2'],
            'min_samples_leaf': [*range(1, 10+1, 2), *range(20, 50+1, 5)],
            'min_samples_split': [*range(1, 10+1, 2), *range(15, 35+1, 5)]
        }
    },
    'gbc': {
        'clf': GradientBoostingClassifier(),
        'grid': {
            'learning_rate': [0.01, 0.05, 0.1, 0.2],
            'n_estimators': [10, 100, 250, 1000],
            'min_samples_split': [100, 250, 500, 1000],
            'max_features': ['auto', 'log2', 'sqrt'],
            'max_depth': [*range(5, 8+1)],
            'subsample': [0.5, 0.7, 1],
        }
    },
    'ada': {
        'clf': AdaBoostClassifier(),
        'grid': {
            'n_estimators': [10, 50, 100, 500],
            'learning_rate': [0.01, 0.1, 0.5, 1, 2],
            'base_estimator': [DecisionTreeClassifier(max_depth = n)
                                    for n in [*range(1, 16+1)]]    
        }
    },
    'xgb': {
        'clf': XGBClassifier(use_label_encoder=False,
                             eval_metric='logloss'),
        'grid': {
#             'n_estimators': [10, 50, 100, 500],
#             'learning_rate': [.02, .05, .1],
#             'max_depth': [4, 6, 8, 10],
        }
    },
    'bayes': {
        'clf': GaussianNB(),
        'grid': {}
    }
}

This part is about the hyperparameter search. For obvious reasons, we shall skip it.

In [22]:
import pickle
import copy

def load_models():
    return pickle.load(open('sklearn-models', 'rb'))

def save_models(est):
    pickle.save(est, open('sklearn-models', 'wb'))

def search(X_train, y_train):
    MAX_N_ITER = 200
    estimators = copy.deepcopy(base)
    
    for name, params in base.items():
        estimators[name] = {'clf': params['clf']}
        if params['grid'] != {}:
            cv = RandomizedSearchCV(params['clf'], params['grid'],
                                   n_iter=MAX_N_ITER, scoring='roc_auc',
                                   cv=5, n_jobs=-1, verbose=10)
            result = cv.fit(X_train, y_train)
            estimators[name]['result'] = result
            estimators[name]['clf'] = result.best_estimator_

This is where we shall actually train the model.

In [57]:
estimators = base

cache = tempfile.TemporaryDirectory()

clf = Pipeline([
    ('stacked', StackingClassifier(
        estimators=[(name, params['clf'])
                    for name, params in estimators.items()],
        final_estimator=GradientBoostingClassifier(),
        n_jobs=2))
], memory=str(cache))

X_train, y_train, _, _ = split(1)
score = cross_validate(clf, X_train, y_train,
                       cv=5, scoring=['roc_auc'], verbose=10,
                       return_estimator=True)

if 'models' not in vars():
    models = []
models = [*models, score]

[CV]  ................................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/home/talos/devs/venvs/py38/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[CV] .................................. , roc_auc=0.782, total=  19.3s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.3s remaining:    0.0s
/home/talos/devs/venvs/py38/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[CV] .................................. , roc_auc=0.787, total=  19.6s
[CV]  ................................................................


...

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   60.0s remaining:    0.0s
/home/talos/devs/venvs/py38/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[CV] .................................. , roc_auc=0.785, total=  21.9s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  1.4min remaining:    0.0s
/home/talos/devs/venvs/py38/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[CV] .................................. , roc_auc=0.791, total=  23.0s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.7min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.7min finished


For dev purposes, we will present some stats about individual classifiers.

In [ ]:
scores = {name: cross_validate(clf['clf'], X_train, y_train,
                               cv=5, scoring=['roc_auc'])
          for name, clf in estimators.items()}
scores = {**scores, 'stacked': score}

score_list = []
for name, score in scores.items():
    for stat in ['fit_time', 'score_time', 'test_roc_auc']:
        for val in score[stat]:
            score_list.append([name, stat, val])

score_df = pd.DataFrame(data=score_list,
                        columns=['est', 'stat', 'val'])

sns.catplot(data=score_df, x='est', hue='stat', y='val',
            kind='box')

Finally, let us save the predictions to a .csv file.

In [96]:
def save_pred_sk(clf, name):
    pred_df = pd.DataFrame(columns=['id', 'y'])
    pred_df['id'] = test_features.index+1
    pred_df['y'] = clf.predict_proba(test_features)[:,1]
    pred_df.to_csv('pred.{}.csv'.format(name), index=False)

In [26]:
save_pred_sk(clf.fit(X_train, y_train), 'scaledv2')

## Model № 2 (TF)

In [24]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

Let's start with the definition of the model.

In [58]:
EPOCHS = 100
BATCH_SIZE = 2048
DROPOUT = 0.5

def block(x_in, size):
    x = layers.Dense(size)(x_in)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(DROPOUT)(x)
    return x

def create_model(shape):
    x_in = layers.Input(shape=shape)
    x = block(x_in, 2048)
    x_out = layers.Dense(1, activation='sigmoid')(x)
    
    model = keras.Model(inputs=x_in, outputs=x_out)    
    model.compile(
        optimizer=keras.optimizers.Adam(lr=1e-3),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=[keras.metrics.AUC(name='auc')])
    
    return model

We will want to balance the predictors in this particular case.

In [59]:
X_train, y_train, X_post, y_post = split(0.9)
X_train, y_train, X_test, y_test = split(0.8, X_train, y_train)

resample = False

if resample:
    neg_features = X_train[y_train == 0]
    neg_labels = y_train[y_train == 0]
    pos_features = X_train[y_train == 1]
    pos_labels = y_train[y_train == 1]

    def make_ds(features, labels):
        ds = tf.data.Dataset.from_tensor_slices((features, labels))
        ds = ds.shuffle(100000).repeat()
        return ds

    neg_ds = make_ds(neg_features, neg_labels)
    pos_ds = make_ds(pos_features, pos_labels)

    train_ds = tf.data.experimental.\
        sample_from_datasets([neg_ds, pos_ds], weights=[0.5, 0.5])
    train_ds = train_ds.batch(BATCH_SIZE).prefetch(2)
    
    pos = np.sum(y_train)
    total = np.size(y_train)
    neg = total - pos
    resampled_steps_per_epoch = np.ceil(2.0*neg/BATCH_SIZE)
else:    
    train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).cache()
    train_ds = train_ds.batch(BATCH_SIZE).prefetch(2)

val_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).cache()
val_ds = val_ds.batch(BATCH_SIZE).prefetch(2)

Here we train the model.

In [60]:
model = create_model(X_train.shape[1])

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

history = model.fit(
    train_ds,
    epochs=EPOCHS,
#     steps_per_epoch=resampled_steps_per_epoch,
#     callbacks=[early_stopping],
    validation_data=val_ds,
    verbose=1)

Epoch 1/100
8/8 [==============================] - 1s 98ms/step - loss: 0.4752 - auc: 0.5747 - val_loss: 1.0985 - val_auc: 0.6262
Epoch 2/100
8/8 [==============================] - 1s 81ms/step - loss: 0.4111 - auc: 0.6503 - val_loss: 0.5130 - val_auc: 0.7396
Epoch 3/100
8/8 [==============================] - 1s 97ms/step - loss: 0.3912 - auc: 0.6550 - val_loss: 0.4985 - val_auc: 0.7374
Epoch 4/100
8/8 [==============================] - 1s 84ms/step - loss: 0.3765 - auc: 0.6757 - val_loss: 0.3973 - val_auc: 0.7254
Epoch 5/100
8/8 [==============================] - 1s 81ms/step - loss: 0.3743 - auc: 0.6796 - val_loss: 0.4123 - val_auc: 0.7080
Epoch 6/100
8/8 [==============================] - 1s 99ms/step - loss: 0.3706 - auc: 0.6813 - val_loss: 0.5388 - val_auc: 0.6693
Epoch 7/100
8/8 [==============================] - 1s 87ms/step - loss: 0.3640 - auc: 0.6887 - val_loss: 0.6426 - val_auc: 0.6555
Epoch 8/100
8/8 [==============================] - 1s 83ms/step - loss: 0.3632 - auc: 0.68

In [61]:
for i in range(10):
    X, y = choose(0.2)
    display(roc_auc_score(y, model.predict(X)))

0.7759658624421556

0.7787348448423828

0.7584259449848378

0.78082914761996

0.7795726405250503

0.7816499544548053

0.785616947580093

0.7763063651325389

0.7688686917062918

0.7775788244453095

Finally, let us save the predictions to a .csv file.

In [97]:
def save_pred_tf(model, name):
    pred_df = pd.DataFrame(columns=['id', 'y'])
    pred_df['id'] = test_features.index+1
    pred_df['y'] = model.predict(test_features)
    pred_df.to_csv('pred.{}.csv'.format(name), index=False)

In [98]:
save_pred_tf(model, 'tfv2')